In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
#from esda.moran import Moran
from libpysal.weights.contiguity import Queen
import statsmodels.api as sm
import statsmodels.formula.api as smf

/home/sara/.local/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_8063/3622933738.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (c

In [2]:
#path1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\som_adm_ocha_itos_20230308_shp\som_admbnda_adm1_ocha_20230308.shp"
path2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/som_adm_ocha_itos_20230308_shp/som_admbnda_adm1_ocha_20230308.shp"
states_gdf = gpd.read_file(path2)

#file1 = r"C:\Users\PcLaptop\Dropbox (Politecnico Di Torino Studenti)\Untitled Folder\distance_df.csv"
file2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/ACLED_1997-01-01-2023-07-18_Somalia.csv"
dist = pd.read_csv(file2)

#csv1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\df_with_dummies.csv"
csv2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/df_CRU4_lag5.csv"
df_with_dummies = pd.read_csv(csv2)

In [3]:
y_var_name = 'conflicts'
X_var_names = ['TA_lag1','PA_lag5','DL_lag5']

In [4]:
# Regression expression for OLS with dummies

unit_names = df_with_dummies['admin1'].unique().tolist()
unit_names.sort()
unit_names_t = df_with_dummies['month_name'].unique().tolist()
unit_names_mr = (df_with_dummies['admin1'] + df_with_dummies['month_name']).unique().tolist()

lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in X_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
   lsdv_expr = lsdv_expr + ' + ' + dummy_name
for dummy_name_t in unit_names_t[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_t
#for dummy_name_mr in unit_names_mr[:-1]:
   # lsdv_expr = lsdv_expr + ' + ' + dummy_name_mr
#lsdv_expr=lsdv_expr + ' - 1' 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=conflicts ~ TA_lag1 + PA_lag5 + DL_lag5 + Awdal + Bakool + Banadir + Bari + Bay + Galgaduud + Gedo + Hiraan + Lower_Juba + Lower_Shabelle + Middle_Juba + Middle_Shabelle + Mudug + Nugaal + Sanaag + Sool + Togdheer + January + February + March + April + May + June + July + August + September + October + November


In [5]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.477
Method:                 Least Squares   F-statistic:                     166.5
Date:                Tue, 19 Sep 2023   Prob (F-statistic):               0.00
Time:                        12:33:05   Log-Likelihood:                -20774.
No. Observations:                5616   AIC:                         4.161e+04
Df Residuals:                    5584   BIC:                         4.182e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.7708      0.731     